In this notebook, I use single model trained separately for each of 24 test sites first for floors then for coordinates. My model uses fastai.
There is almost no postprocessing except for floor prediction. I corrected floor predictions to make all points being on the same floor (majority of predictions).
All this code (5 folds CV) runs more than 12 hours and I run it locally. So after checking everything works I upload my submission file and will submit it instead of waiting of the whole run here. Also on Kaggle I cannot run on GPU (not enough memory), so use only cpu here.

About the model per se.
It uses RSSI from BSSIDs met more than X times in given site. Threshold is defined for each building. To define coordinates of records with wifi-data we use calibrate_magnetic_wifi_ibeacon_to_position provided by host, fix for new lines used too. But as we add dataset with this information (ie txt files were already parsed) we don't attach fixes for hosts files.

Further details like augmenations I'll clarify below.

I think the score for the single model without ensembles and postprocessing is apropriate so may be it will be useful for someone.

Thank you

In [ ]:
import os.path as op
import sys
import glob
import pandas as pd
import numpy as np
import json
from IPython.core.debugger import Tracer
import os
import matplotlib.pyplot as plt
import time
import re
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder

from fastai.tabular.all import *
import random
import datetime
from sklearn.model_selection import StratifiedKFold

In [ ]:
CSV_PATH = "../input/indoor-generated" #this is for csv files which I generated before, If you wish regenerate - handle this to see test and train txt
TXT_PATH = "../input/indoor-location-navigation"
OUTPUT = ""
#TEST_PATH = op.join(TXT_PATH, "test") 
#TRAIN_PATH = op.join(TXT_PATH, "train")
#META_PATH = op.join(PATH, "metadata")
#SCRIPTS_PATH = op.join(PATH, "scripts", "indoor-location-competition-20") #see comments above
SS_PATH = op.join(TXT_PATH, "sample_submission.csv")
TRAIN_CSV_PATH = op.join(CSV_PATH, "train_csv", "train_csv")
TEST_CSV_PATH = op.join(CSV_PATH, "test_csv", "test_csv")
#TEST_ALL_PATHS_CSV = op.join(TEST_CSV_PATH, "test_all_paths.csv")
MODEL_DIR = op.join(OUTPUT, "models")
#CLOSEST_WAYPOINT_CSV_DIR = op.join(TRAIN_CSV_PATH, "train_df_closest_waypoint")
TODAY = datetime.date.today().strftime("%d%m%Y")
#os.mkdir(op.join(PATH, TODAY))
#sys.path.append(SCRIPTS_PATH) 
#from main import calibrate_magnetic_wifi_ibeacon_to_position
sys.path.append("../input/iterative-stratification/iterative-stratification-master") 
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
# from io_f import read_data_file
# from compute_f import compute_step_positions
# from compute_f import compute_steps
# from compute_f import compute_headings
# from compute_f import compute_stride_length
# from compute_f import compute_step_heading
# from compute_f import compute_rel_positions
# from compute_f import correct_positions
# from compute_f import correct_trajectory
!dir $CSV_PATH


In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [ ]:
ss_df = pd.read_csv(SS_PATH)
ss_df_tmp = ss_df['site_path_timestamp'].apply(lambda x: pd.Series(x.split("_")))
ss_df_tmp = ss_df_tmp.rename(columns={0: "site", 1: "path", 2:"timestamp"})
ss_df_tmp['timestamp'] = pd.to_numeric(ss_df_tmp['timestamp'])
test_sites = np.unique(ss_df_tmp.site.values)#TODO will train only with sites from test, check later if we adding others sites can help

Below we generate csv files from txt for training and prediction. But as we added dataset already, will skip this. I got no results with IBeacon so I don't used it.

In [ ]:
def get_df_from_file(file):
    data = calibrate_magnetic_wifi_ibeacon_to_position([file])
    
    wifi_dfs = []
    ibeacon_dfs = []
     
    for key in data:
        x,y = key       
               
        if ('wifi' in data[key] and len(data[key]['wifi']) > 0):
            wifi_df = pd.DataFrame(columns=['path','floor','timestamp','x','y','bssid','rssi', 'lastseen'])
            wifi_data = data[key]['wifi']             
            wifi_df['timestamp'] = wifi_data[:,0].astype(float)
            wifi_df['bssid'] = wifi_data[:,2]
            wifi_df['rssi'] = wifi_data[:,3].astype(float)
            wifi_df['lastseen'] = wifi_data[:,4].astype(float)
            wifi_df['x'] = x
            wifi_df['y'] = y   #next time add last seen
            wifi_dfs.append(wifi_df)
        elif ('ibeacon' in data[key] and len(data[key]['ibeacon']) > 0):   
            ibeacon_df = pd.DataFrame(columns=['path', 'floor', 'timestamp', 'uuid', 'major', 'minor', 'txpower',
                                               'rssi', 'distance', 'mac', 'lastseen'])
          
            ibeacon_data = data[key]['ibeacon'] 
            ibeacon_df['timestamp'] = ibeacon_data[:,0].astype(float)
            ibeacon_df['uuid'] = ibeacon_data[:,1]
            ibeacon_df['major']= ibeacon_data[:,2]
            ibeacon_df['minor'] = ibeacon_data[:,3]
            ibeacon_df['txpower'] = ibeacon_data[:,4].astype(float)
            ibeacon_df['rssi'] = ibeacon_data[:,5].astype(float)
            ibeacon_df['distance'] = ibeacon_data[:,6].astype(float)
            ibeacon_df['mac'] = ibeacon_data[:,7]
            ibeacon_df['lastseen'] = ibeacon_data[:,8].astype(float)
            ibeacon_df['x'] = x
            ibeacon_df['y'] = y
            ibeacon_dfs.append(ibeacon_df)
           
    return (pd.concat(wifi_dfs) if len(wifi_dfs) > 0 else None),\
           (pd.concat(ibeacon_dfs) if len(ibeacon_dfs) > 0 else None)

def generate_pre_csv(generate=False):
    if not generate:
        return    

    for i in range(1,len(test_sites)):
        site = test_sites[i]
        print(i, site)
        floor_files = glob.glob(op.join(TRAIN_PATH, site,'*'))  

        floor_wifi_dfs = []
        floor_ibeacon_dfs = []
        
        for floor_file in floor_files:
            
            floor = floor_file.split("\\")[-1]
            files = glob.glob(op.join(floor_file, "*"))

            for file in files:
                path = file.split("\\")[-1][:-4]
                #print(file)
               
                wifi_df, ibeacon_df = get_df_from_file(file)

                if (wifi_df is not None):
                    wifi_df['path']=path
                    wifi_df['floor']=floor                     
                    floor_wifi_dfs.append(wifi_df)    
                
                if (ibeacon_df is not None):
                    ibeacon_df['path']=path
                    ibeacon_df['floor']=floor
                    floor_ibeacon_dfs.append(ibeacon_df)
            #break
        site_wifi_df = pd.concat(floor_wifi_dfs)
        site_wifi_df.to_csv(op.join(TRAIN_CSV_PATH, site + '.wifi.pre.csv'),index=False)  
        
        site_ibeacon_df = pd.concat(floor_ibeacon_dfs)
        site_ibeacon_df.to_csv(op.join(TRAIN_CSV_PATH, site + '.ibeacon.pre.csv'),index=False)  
        #break
        
def generate_test_pre_csv(generate=False):    
    if not generate:
        return    

    for i in range(len(test_sites)):
        site = test_sites[i]
        print(i, site)
        paths = ss_df_tmp[ss_df_tmp['site']==site]['path'].values
        wifi_paths_df = []
        ibeacon_paths_df = []
        
        for path in paths:
            file = op.join(TEST_PATH, path + ".txt")
            data = read_data_file(file)
            
            wifi_path_df = pd.DataFrame(columns=['timestamp', 'ssid', 'bssid', 'rssi', 'lastseen'])            
            wifi_data = data.wifi
            for i in range(len(wifi_path_df.columns)):
                wifi_path_df[wifi_path_df.columns[i]] = wifi_data[:,i]
            wifi_path_df['path'] = path
            wifi_paths_df.append(wifi_path_df)
            
            ibeacon_path_df = pd.DataFrame(columns=['timestamp', 'uuid', 'major', 'minor', 'txpower',
                                                    'rssi', 'distance', 'mac', 'lastseen'])            
            ibeacon_data = data.ibeacon
            if (len(ibeacon_data) > 0):
                for i in range(len(ibeacon_path_df.columns)):
                    ibeacon_path_df[ibeacon_path_df.columns[i]] = ibeacon_data[:,i]
                ibeacon_path_df['path'] = path
            ibeacon_paths_df.append(ibeacon_path_df)
            
                       
            
        wifi_site_df = pd.concat(wifi_paths_df)
        wifi_site_df.to_csv(op.join(TEST_CSV_PATH, site + '.wifi.pre.csv'),index=False)  
        
        ibeacon_site_df = pd.concat(ibeacon_paths_df)
        ibeacon_site_df.to_csv(op.join(TEST_CSV_PATH, site + '.ibeacon.pre.csv'),index=False)  
       
print("generating train pre csv")       
generate_pre_csv(generate=False)
print("generating test pre csv")       
generate_test_pre_csv(False)

In [ ]:
#So for now see no good from choosing the closest wifi records

def generate_wifi_files(generate = False):
    if (not generate):
        return
    for i in range(14,len(test_sites)):
        site = test_sites[i]
        print(i, site)
        df = pd.read_csv(op.join(TRAIN_CSV_PATH, site + ".wifi.pre.csv"))#TODO store with index=False      
        
        bssids = list(np.unique(df['bssid'].values))
        train_df = pd.DataFrame(columns=['timestamp','x','y','floor','path'] + bssids)
        for (i,(a,b)) in tqdm(enumerate(df.groupby(['timestamp','x','y']))):
            #v = b['time_diff']
            
            timestamp, x, y = a
            bssid_values = b['bssid'].values    
            rssi_values = b['rssi'].values            
            row = {'timestamp':timestamp, 'x':x, 'y':y}
            row.update({k:v for (k,v) in zip(bssid_values,rssi_values)})    
            train_df = train_df.append(row, ignore_index=True)
            
        for (i,(a,b)) in tqdm(enumerate(df[['timestamp', 'floor', 'path']].groupby(['timestamp']))):
            timestamp  = a
            floor = b['floor'].values[0]
            path = b['path'].values[0]
            
            train_df.loc[train_df[train_df['timestamp']==a].index,['floor', 'path']] = [floor, path]
            
        train_df = train_df.fillna(0)        
        
        train_df.to_csv(op.join(TRAIN_CSV_PATH, site + ".wifi.csv"), index=False) 

def generate_ibeacon_files(generate = False):
    if (not generate):
        return        
        
    for i in range(len(test_sites)):
        site = test_sites[i]
        print(i, site)
        df = pd.read_csv(op.join(TRAIN_CSV_PATH, site + ".ibeacon.pre.csv"))#TODO store with index=False      
        
        macs = list(np.unique(df['mac'].values))
        rssi_cols = ['rssi_' + mac for mac in macs]
        txp_cols = ['txp_' + mac for mac in macs]
        dist_cols =  ['dist_' + mac for mac in macs]
        cols = rssi_cols + txp_cols + dist_cols
        
        train_df = pd.DataFrame(columns=['timestamp','x','y','floor','path'] + cols)
        for (i,(a,b)) in tqdm(enumerate(df.groupby(['timestamp','x','y']))):
                     
            timestamp, x, y = a
            mac_values = b['mac'].values
            rssi_values = b['rssi'].values
            dist_values = b['distance'].values
            txpower_values = b['txpower'].values
            
                      
            row = {'timestamp':timestamp, 'x':x, 'y':y}
            row.update({'rssi_'+ k:v for (k,v) in zip(mac_values,rssi_values)})   
            row.update({'txpower_'+ k:v for (k,v) in zip(mac_values,txpower_values)})
            row.update({'dist_'+ k:v for (k,v) in zip(mac_values,dist_values)})
            
            train_df = train_df.append(row, ignore_index=True)
            
        for (i,(a,b)) in tqdm(enumerate(df[['timestamp', 'floor', 'path']].groupby(['timestamp']))):
            timestamp  = a
            floor = b['floor'].values[0]
            path = b['path'].values[0]
            
            train_df.loc[train_df[train_df['timestamp']==a].index,['floor', 'path']] = [floor, path]
            
        train_df = train_df.fillna(0)        
        
        train_df.to_csv(op.join(TRAIN_CSV_PATH, site + ".ibeacon.csv"), index=False)        

print("Will generate train csv")            
generate_wifi_files(False) 
print("Will generate ibeacon csv")
generate_ibeacon_files(False)

So next goes my main cell.

In [ ]:
EPOCHS = 1 # I used 20 epochs !!!!! this is for quick run
FOLDS = 5

# As RSSI is always negative and zero in my csv means no signal there is a contradiction. Zero is much more than very strong signal. So I move 
# RSSI to be above zero, so 0 means very bad signal. Than I factorize by 2.5 (the best value for me, round and /5 was worse/.)
def handle_values(df):
    needed_cols = [c for c in df.columns if len(c) > 15]
    values = df[needed_cols].values    
    values=np.where(values==0, -100, values)
    values = np.maximum(0, values + 90)
    df[needed_cols] = np.round(values/2.5)
    
    return df
#these my thresholds for different sites (how often bssid should be met to participate in train)
thr_dict = {
    test_sites[0]:200,
    test_sites[1]:500,
    test_sites[2]:500,
    test_sites[3]:250,
    test_sites[4]:200,
    test_sites[5]:150,
    
    test_sites[6]:200,
    test_sites[7]:250,
    test_sites[8]:250,
    test_sites[9]:200,
    test_sites[10]:250,
    test_sites[11]:500,
    
    test_sites[12]:250,
    test_sites[13]:200,
    test_sites[14]:250,
    test_sites[15]:200,
    test_sites[16]:500,
    test_sites[17]:200,
    
    test_sites[18]:250,
    test_sites[19]:150,
    test_sites[20]:200,
    test_sites[21]:500,
    test_sites[22]:500,
    test_sites[23]:400##200  
    
    
}

# here we compute which columns should be used (based on threshold), make fold stratification and one-hot encoding
def get_train_df(site, pixel_strat=True, do_thr=True, wifi=True):
    suffix = ".wifi.csv" if wifi else '.ibeacon.csv'
    train_df = pd.read_csv(op.join(TRAIN_CSV_PATH, site + suffix))
    train_df = handle_values(train_df)
    enc = OneHotEncoder(sparse=False)
    result = enc.fit_transform(train_df['floor'].values.reshape(-1,1))
    cat_names = enc.categories_[0]
    train_df[cat_names] = result    
    
    train_df['kfold'] = -1
    
    
    if (pixel_strat):        
        kf = MultilabelStratifiedKFold(n_splits=FOLDS)#, random_state=12)
        y = train_df[enc.categories_[0]].values
        for fold, (t_,v_) in enumerate(kf.split(X=train_df,y=y)):
            train_df.loc[v_,'kfold'] = fold
    else:
        skf = StratifiedKFold(n_splits=5)#, random_state=12)
        help_df = train_df[['path','floor']]
        help_df = help_df.drop_duplicates(subset=['path', 'floor'])
        y_for_split = help_df['floor'].values
        for fold, (t_,v_) in enumerate(skf.split(help_df, y_for_split)):
            for p, f in help_df.iloc[v_][['path','floor']].values:
                index = train_df[(train_df['path']==p) & (train_df['floor']==f)].index
                train_df.loc[index, 'kfold']= fold
    
    columns = [c for c in train_df.columns if len(c) > 20]
    if (do_thr):
        thr = thr_dict[site]#min(thr_dict[site],250)#thr_dict[site]
        
        print("threshold: ", str(thr))
        cont_names = [c for c in columns if train_df[c].ne(0).sum() > thr]
    else:
        cont_names = columns
    return train_df, list(enc.categories_[0]), cont_names, columns

#just return floor for submission by letter
def get_floor(floor_str):
    num = re.search('\d+',floor_str)[0]
    if ('F'  in floor_str):
        return int(num)-1
    elif ('B'  in floor_str):
        return -int(num)
    else:
        raise ValueError('A very specific bad thing happened.')  
        
#Augmentation is very simple:  sometimes some of wifi  points can be switche off, so we replace true RSSI by zero.    
class AugCallback(Callback):
    #run_after,run_valid = [Normalize],False
    #run_before = [Normalize]
    #def __init__(self, alpha=0.4): self.distrib = Beta(tensor(alpha), tensor(alpha))
    
    def before_batch(self, **kwargs):      
        #Tracer()()
        ret_condition = (self.learn.dls[0] != self.learn.dl)
        
        if (ret_condition): #if not train do nothing
            return 
        
        cat,x = self.xb
        #y,_ =self.learn.yb        
        non_zeros_counts = self.xb[1].count_nonzero(dim=1)
        make_zero_counts = (self.xb[1].count_nonzero(dim=1)/50).round()
        changed_counts = (self.xb[1].count_nonzero(dim=1)/50).round()
        bs = x.shape[0]
        for i in range(bs):
            elem = x[i]
            #Tracer()()
            indexes = elem.nonzero().reshape(-1)
            for_zero_indexes = np.random.randint(0, len(indexes), int(make_zero_counts[i]))
            change_count_i = int(changed_counts[i])
            for_change_indexes = np.random.randint(0, len(indexes), change_count_i)
            x[i, indexes[for_zero_indexes]]=0
            #Tracer()()
            #factor =  torch.tensor(0.95 + 0.1*np.random.rand(change_count_i)).to(x.device)
            #x[i, indexes[for_change_indexes]] *=factor
            
        #Tracer()()
        self.learn.xb = (cat, x)
        #print(a.shape, b.shape)   
        

#Nothing special, cat names is empty, experiments with categories were not successful :)
def get_data(train_df,
             fold, target_names = [], 
             procs = [Categorify, FillMissing],#, Normalize],
             cat_names = [],
             cont_names = []):
    val_idx = train_df[train_df.kfold==fold].index
    
    dls = TabularDataLoaders.from_df(train_df, path=CSV_PATH, 
                                        y_names=target_names,#['B1', 'F1', 'F2', 'F3', 'F4'],list(enc.categories_[0])
                                        cat_names = cat_names,
                                        
                                        cont_names = cont_names,
                                        procs = procs,#, Normalize],
                                        valid_idx=val_idx,
                                        bs=64)
    return dls



def run_train(train_df, fold, test_df, y_names, cat_names, cont_names, loss_func, y_range, name):
    #Tracer()()
    #new = train_df.loc[train_df['floor']=='B1'].reset_index()
    dls = get_data(train_df, fold, target_names=y_names,cat_names=cat_names, cont_names= cont_names) # Data
    model_dir = '/kaggle/working/'
    learn = tabular_learner(dls, 
                        y_range=y_range,
                            layers = [1024, 512, 512, 256],     
                            #layers = [2048, 1024, 512, 256], 
                            #layers = [1536, 1024, 512, 256],
                           
                                #loss_func = BCELossFlat(),
                                loss_func = loss_func,
                                #config=config,
                                model_dir=model_dir,
                            #cbs=cbs
                                cbs= [AugCallback()]
                   ) # Model
    cb = SaveModelCallback(monitor='valid_loss',fname=name )
    lr=1e-3   
    learn.fit_one_cycle(EPOCHS, lr_max=slice(lr/(2.6**4),lr)) # Training
    lr=1e-4
    learn.fit_one_cycle(EPOCHS, lr_max=slice(lr/(2.6**4),lr), cbs = cb) # Training  
    
    learn.load(name)
    test_dl = learn.dls.test_dl(test_df)#learn.dls.valid#learn.valid_dllearn.dls.test_dl(test_features)
    test_preds, _ = learn.get_preds(dl=test_dl) # prediction
    #Tracer()()
    
    #test_dl = learn.dls.test_dl(test_df)#learn.dls.valid#learn.valid_dllearn.dls.test_dl(test_features)
    val_preds, val_labels = learn.get_preds(dl=learn.dls.valid) # prediction
    return val_preds, val_labels, test_preds


def get_test_df(site, columns,generate = False, wifi = False):
    suffix = ".wifi.csv" if wifi else '.ibeacon.csv'
    if (not generate):
        df = pd.read_csv(op.join(TEST_CSV_PATH, site + suffix))
        df = handle_values(df)
        return df
    suffix2 = ".wifi.pre.csv" if wifi else '.ibeacon.pre.csv'
    df = pd.read_csv(op.join(TEST_CSV_PATH, site + suffix2))
    result_df = pd.DataFrame(columns = ['timestamp', 'path'] + columns)
    for (i,(a,b)) in tqdm(enumerate(df.groupby(['timestamp']))):
        timestamp= a
        row = {'timestamp':timestamp}
        if (wifi):
            bssid_values = b['bssid'].values    
            rssi_values = b['rssi'].values           
            row.update({k:v for (k,v) in zip(bssid_values,rssi_values)})                
        else:
            mac_values = b['mac'].values
            rssi_values = b['rssi'].values
            dist_values = b['distance'].values
            txpower_values = b['txpower'].values                    
            
            row.update({'rssi_'+ k:v for (k,v) in zip(mac_values,rssi_values)})   
            row.update({'txpower_'+ k:v for (k,v) in zip(mac_values,txpower_values)})
            row.update({'dist_'+ k:v for (k,v) in zip(mac_values,dist_values)})
        result_df = result_df.append(row, ignore_index=True)     
            
    for (i,(a,b)) in tqdm(enumerate(df[['timestamp', 'path']].groupby(['timestamp', 'path']))):
        timestamp, path  = a
      
        #path = b['path'].values[0]

        result_df.loc[result_df[result_df['timestamp']==timestamp].index,['path']] = path


    result_df = result_df.fillna(0)        
    result_df.to_csv(op.join(TEST_CSV_PATH, site  + suffix), index=False)  
    result_df = handle_values(result_df)
    return result_df

def update_ss_df(test_df, f_test_preds, xy_test_preds, site, cat_names):
    floors_ind = np.argmax(f_test_preds, axis=1)
    test_df['floor'] = floors_ind.tolist()
    test_df['x'] = xy_test_preds[:,0].tolist()
    test_df['y'] = xy_test_preds[:,1].tolist()
    
    for path, ss_time in ss_df_tmp[ss_df_tmp['site']==site][['path', 'timestamp']].values:
        timestamps = test_df[test_df['path'] == path]['timestamp'].values
        distances = np.abs(timestamps - ss_time)
        argmin = np.argmin(distances)
        #print("min dist found", distances[argmin])
        timestamp = timestamps[argmin]
        #result['floor'] = [get_floor(cat_names[i]) for i in np.argmax(e,axis=1)]
        #Tracer()()
        row = test_df[(test_df['path']==path)&(test_df['timestamp']==timestamp)]
    
        floor = get_floor(cat_names[row['floor'].values[0]])
        xy = row[['x','y']].values[0]
        record_id = site + "_" + path + "_" + str(ss_time+ 10000000000000)[1:]
        index = ss_df[ss_df['site_path_timestamp']==record_id].index    
        ss_df.loc[index, ['floor','x','y']] = [floor, xy[0], xy[1]]
        
valid_dict = {}

def fix_floor_in_df(df):
    for a,b in df[['site', 'floor','path']].groupby(['site', 'path']):
        site, path = a
        floors = b['floor']
        
        uniques = np.unique(floors)
        if (len(uniques) > 1):
            #Tracer()()
            print("site: ", site, ", path:", path)
            ind = np.argmax([np.where(floors==elem, 1, 0).sum() for elem in uniques])
            floor = uniques[ind]
            df_index = df[(df['path']==path)&(df['site']==site)].index
            df.loc[df_index, 'floor']=floor
            print(floors)
            print("!!!", floor)

    for a,b in df[['site', 'floor','path']].groupby(['site', 'path']):
        site, path = a
        floors = b['floor']

        uniques = np.unique(floors)
        if (len(uniques) > 1):
            #Tracer()()        
            print("Not fixed", floors)
    return df

PREFIX = TODAY
  
for i, site in enumerate(test_sites):
    #seed_everything(i) 
    xy_test_all = []
    floor_test_all = []
    print("_______Staring", i, " of ", len(test_sites), " sites")
    
    train_df, cat_names,cont_names, columns = get_train_df(site, True, True, True)
    test_df = get_test_df(site, columns, False, True)
    print(i, " ", len(cont_names), " ",len(train_df), " ")#, len(test_site_df))
        #TODO add number of folds
    
    for fold in range(FOLDS):
        print("Starting fold ", fold)
        seed_everything(fold)         

        f_val_preds, f_val_labels, f_test_preds = run_train(train_df, fold, test_df, cat_names, [],cont_names, BCELossFlat(),
                                                             (0,1),site + "_" + str(fold) + "_f")
        
        #f_val_preds, f_val_labels, f_test_preds = run_train(train_df, fold, test_df, cat_names, [],cont_names, None,
        #                                                    None,site + "_" + str(fold) + "_f")

        valid_dict[site + "_f_labels"] = f_val_labels
        valid_dict[site + "_f_preds"] = f_val_preds
        
        floor_test_all.append(f_test_preds.tolist())
   
    
    
#     for fold in range(5):
#         print("Starting fold for xy", fold)
#          seed_everything(fold)

    
    
    
        if i == 1 or i == 20 or i ==22:
            width = 500
        else:
            width = 350
        
        xy_val_preds, xy_val_labels, xy_test_preds = run_train(train_df, #train_df[train_df[(train_df['B1'] != 1) & (train_df['B2'] != 1)]],

                                                               fold, test_df, ['x','y'], [],cont_names, MSELossFlat(),
                                                               (0, width), site + "_" + str(fold) + "_xy") 
        
        xy_test_all.append(xy_test_preds.tolist())
    
    xy_test_all = np.array(xy_test_all).mean(axis=0)  
    test_df['x'] = xy_test_all[:,0]
    test_df['y'] = xy_test_all[:,1]
    floor_test_all = np.array(floor_test_all).mean(axis=0)
    test_df['floor'] = [cat_names[f] for f in floor_test_all.argmax(axis=1).tolist()]
    test_df['site'] = site
    
    #test_df.to_csv(op.join(PATH, PREFIX, "test_df_" + site + ".csv"))
    
    test_df = fix_floor_in_df(test_df)
    update_ss_df(test_df, floor_test_all, xy_test_all, site, cat_names) 
    valid_dict[site + "_xy_labels"] = xy_val_labels
    valid_dict[site + "_xy_preds"] = xy_val_preds
    
    #ss_df.to_csv(op.join(PATH, PREFIX, "ss_df_" + site + ".csv"))

In [ ]:

ss_df_tmp['floor'] = ss_df['floor']
ss_df_tmp = fix_floor_in_df(ss_df_tmp)           
ss_df['floor']=ss_df_tmp['floor']           
ss_df.to_csv(op.join("", "submission-generated.csv"), index=False)

##submission to don't wait long run
#This file was gotten on my local machine with EPOCHS=20
subm = pd.read_csv(op.join(CSV_PATH, "04.05.1.csv"))
subm.to_csv(op.join("", "subm.csv"), index=False)